In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-cbc82f0b-9a6b-eeb7-bdba-fcdd873f74d4)


### Code for all important tasks

In [2]:
import tensorflow as tf

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

# Note: The following confusion matrix code is a remix of Scikit-Learn's
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Plot the validation and training data separately
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.

    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here)
    """

    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

# Create function to unzip a zipfile into current working directory
# (since we're going to be downloading and unzipping a few files)
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

### Get the data

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-06-18 01:44:23--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.207, 142.250.99.207, 142.250.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-06-18 01:44:23 (108 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing the text data

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head() #1 is disaster, 0 is not

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42) # frac for what percent of data (here 1 means 100%)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# How many samples of each class?
train_df_shuffled.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [7]:
# visualize some random data
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(disaster)" if target > 0 else "(not disaster)")
  print(f"Text:\n{text}\n")

Target: 1 (disaster)
Text:
She didn't even notice me in the background I'm a floatin head #bombed http://t.co/JA0WGp8sPe

Target: 1 (disaster)
Text:
Student electrocuted to death in school campus http://t.co/ryah8Fni5Q

Target: 0 (not disaster)
Text:
Electric vs Gas brewing (not wanting to electrocute myself) question http://t.co/26oo0fcL53

Target: 0 (not disaster)
Text:
A whistleblower lawsuit accuses that supervisor of sleeping on the job more than once according to officials. 

http://t.co/IPwySnik0G

Target: 0 (not disaster)
Text:
#PBBan (Temporary:300) avYsss @'aRmageddon | DO NOT KILL | FLAGS ONLY | Fast XP' for Reason



### Split into train and validation tests

In [8]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [9]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [10]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

### Need to convert texts into numbers.
Tokenization vs. Embedding
1. Tokenization - every number will be labelled(direct mapping) from 0 to end(like how one-hot encoding works) but its too time taking and complex for texts.
2. Embedding - Every words will be converted into vectors. Matrix of feature vectors. Depending on size be 128 dimensional vector, 512 vector etc, also gets updated every time neural network keeps learning. Can be created or use pre-learned embedding, like Word2Vec, GloVe etc.

### 1. Text Vectorization(Tokenization)
1. Standardize each sample(lowercase + punctuation stripping)
2. Split each sample into substrings(usually words)
3. recombine substrings into tokens(ngrams)
4. index tokens
5. transform using index into vectors

In [11]:
# Text Vectorization(Tokenization)
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [12]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#use the default variables
text_vectorizer = TextVectorization(max_tokens=None, #Here None means no cap, that there is no particular number of occurences specified for any word
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, #treat every token of its own, instead of groups
                                    output_mode='int', #maps token into integer vectors
                                    output_sequence_length=None) #shorter words gets padded to zero, if None according to the max length


In [13]:
#average number of words in the sentences
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
# Setup custom text vectorization variables
max_vocab_length = 10000 #max number of words to have in the vocabulary
max_length = 15 #max length the sequences will be

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [15]:
#Fit the text vectorizer to the training sentences
text_vectorizer.adapt(train_sentences)

In [16]:
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n{random_sentence}\
      \n\nVectorized version of the text:")
text_vectorizer([random_sentence])

Original Text:
when you're taking a shower and someone flushes the toilet and you have .1 second to GTFO or you get burned??????????????????????????????????????????????????      

Vectorized version of the text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  45,  172,  917,    3, 1593,    7,  475,    1,    2, 1749,    7,
          12,   24,  198,  642]])>

In [17]:
# get the unique words in vocabulary after its fit for tokenizer
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocabulary: {len(words_in_vocab)}")
print(f"Most common words: {top_5_words}")
print(f"Least common words: {bottom_5_words}")

Number of words in vocabulary: 10000
Most common words: ['', '[UNK]', 'the', 'a', 'in']
Least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### 2. Embedding Layer

Tensorflow's predefined embedding layer-
* `input_dim` - size of vocabulary
* `output_dim` - size of output embedding layer as matrix
* `embeddings_initializer` - length of sequences being passed
* `input_length` - length of sequences being passed to embedding layer

In [18]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length #how long is each input
                             )
embedding

In [19]:
random_sentence = random.choice(train_sentences)

print(f"Original Text:\n{random_sentence}\
      \n\nEmbedded version of the text:\n")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
Fun with my girls! @ Joe Cools - Durban Main Page https://t.co/AbnZQWlig1      

Embedded version of the text:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02482028,  0.00540805,  0.04710258, ...,  0.02030443,
         -0.04956359, -0.0042346 ],
        [ 0.01383263, -0.04296986,  0.01006452, ..., -0.03909538,
         -0.0183437 , -0.03951347],
        [ 0.01287855,  0.01747819,  0.00936042, ...,  0.04471884,
          0.04098639, -0.02220879],
        ...,
        [-0.04101186, -0.04695866, -0.01696306, ..., -0.02654954,
          0.04794414,  0.02718303],
        [-0.04101186, -0.04695866, -0.01696306, ..., -0.02654954,
          0.04794414,  0.02718303],
        [-0.04101186, -0.04695866, -0.01696306, ..., -0.02654954,
          0.04794414,  0.02718303]]], dtype=float32)>

In [20]:
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02482028,  0.00540805,  0.04710258, -0.04992056,  0.01669325,
         0.01735356,  0.03685391,  0.03674889, -0.03683367,  0.03846813,
         0.03899903, -0.01576173,  0.01883224, -0.03646929,  0.01840134,
        -0.0179017 , -0.0286178 ,  0.03912879,  0.02772423, -0.03498568,
        -0.02838796,  0.04420933, -0.01443292, -0.04469247, -0.03063156,
         0.01550475,  0.01222429, -0.03367941, -0.0231346 ,  0.03188555,
         0.02844549,  0.00779712, -0.02435106,  0.00979616, -0.02787334,
        -0.04079838, -0.00350986, -0.02611244,  0.02141738, -0.03417277,
         0.00935019,  0.03459303,  0.04888475,  0.00513021,  0.0322541 ,
        -0.03126131,  0.02136041,  0.02441852,  0.02375722,  0.01347089,
        -0.01038591,  0.02576501, -0.02427329, -0.03832306, -0.02827784,
         0.01930299,  0.02750901, -0.03026344,  0.01667653, -0.01628898,
        -0.00590821,  0.02209875,  0.00101095, -0.00762124,  0.02071613,
  

### Model 0
Baseline model with Naive Bayes with TF-IDF to convert words to numbers

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Make pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), #converts words to numbers
    ("clf", MultinomialNB()) #model the text
])

#Fit the pipeline on the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [22]:
#Evaluate the model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Model Accuracy: {baseline_score*100:.2f}%")

Baseline Model Accuracy: 79.27%


In [23]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

Evaluation metrics

In [24]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [25]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1
FNN model - A simple dense model

In [26]:
#Directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [27]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") #inputs are 1 dimensional strings

x = text_vectorizer(inputs) #Turn the input texts into numbers
x = embedding(x) #create embedding of the numbers

#x = layers.Flatten()(x) #to reshape to the size of labels
x = layers.GlobalAveragePooling1D()(x) #reduce sequence dimension to match the shape of labels

outputs = layers.Dense(1, activation='sigmoid')(x) #sigmoid activation for binary

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')


In [28]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [29]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [30]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data =(val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                      experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240618-014435
Epoch 1/10
215/215 [==============================] - 22s 69ms/step - loss: 0.6113 - accuracy: 0.6898 - val_loss: 0.5353 - val_accuracy: 0.7559
Epoch 2/10
215/215 [==============================] - 2s 9ms/step - loss: 0.4414 - accuracy: 0.8167 - val_loss: 0.4722 - val_accuracy: 0.7822
Epoch 3/10
215/215 [==============================] - 2s 7ms/step - loss: 0.3467 - accuracy: 0.8629 - val_loss: 0.4649 - val_accuracy: 0.7913
Epoch 4/10
215/215 [==============================] - 2s 7ms/step - loss: 0.2847 - accuracy: 0.8913 - val_loss: 0.4659 - val_accuracy: 0.7927
Epoch 5/10
215/215 [==============================] - 1s 5ms/step - loss: 0.2375 - accuracy: 0.9133 - val_loss: 0.4804 - val_accuracy: 0.7808
Epoch 6/10
215/215 [==============================] - 1s 7ms/step - loss: 0.2011 - accuracy: 0.9288 - val_loss: 0.5015 - val_accuracy: 0.7900
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss

In [31]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.6227 - accuracy: 0.7743


[0.6226829886436462, 0.7742782235145569]

In [32]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [33]:
model_1_pred_probs[:10]

array([[0.41217178],
       [0.6957635 ],
       [0.99949753],
       [0.0597903 ],
       [0.00716836],
       [0.98317873],
       [0.87575996],
       [0.9996215 ],
       [0.9983639 ],
       [0.49559468]], dtype=float32)

In [34]:
#Convert model prediction probabilities to label format so <0.5 is 0(not disaster) and >0.5 is 1(disaster)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
model_1_results = calculate_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 77.42782152230971,
 'precision': 0.7759894665484696,
 'recall': 0.7742782152230971,
 'f1': 0.7722311836526509}

In [36]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [37]:
# function to compare baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 77.43, Difference: -1.84
Baseline precision: 0.81, New precision: 0.78, Difference: -0.04
Baseline recall: 0.79, New recall: 0.77, Difference: -0.02
Baseline f1: 0.79, New f1: 0.77, Difference: -0.01


### Visualize learned embeddings

In [38]:
#Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:5]

(10000, ['', '[UNK]', 'the', 'a', 'in'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [40]:
#Get the weight matrix of the embedding layer
embed_weights = model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [41]:
# # # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0:
#      continue # skip padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

### RNNs are useful for sequence data.
Basically, use information from the past to help with the future (this is where the term recurrent comes from). In other words, take an input (X) and compute an output (y) based on all previous inputs.

variants of the RNN:

* Long short-term memory cells (LSTMs).
* Gated recurrent units (GRUs).
* Bidirectional RNN's (passes forward and backward along a sequence, left to right and right to left).

### Model 2: LSTM
Long Short Term Memory
* Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)

The main difference will be that we're going to add an LSTM layer between our embedding and output.

And to make sure we're not getting reusing trained embeddings (this would involve data leakage between models, leading to an uneven comparison later on), create another embedding layer (model_2_embedding) for model. The text_vectorizer layer can be reused since it doesn't get updated during training.

In [42]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers

model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
x = layers.LSTM(64)(x) # return vector for whole sequence
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [43]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [44]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                      

In [45]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data= (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 'LSTM')])

Saving TensorBoard log files to: model_logs/LSTM/20240618-014522
Epoch 1/10
215/215 [==============================] - 11s 34ms/step - loss: 0.5162 - accuracy: 0.7376 - val_loss: 0.4555 - val_accuracy: 0.7822
Epoch 2/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3199 - accuracy: 0.8689 - val_loss: 0.5007 - val_accuracy: 0.7782
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.2198 - accuracy: 0.9136 - val_loss: 0.5782 - val_accuracy: 0.7769
Epoch 4/10
215/215 [==============================] - 2s 8ms/step - loss: 0.1540 - accuracy: 0.9421 - val_loss: 0.6703 - val_accuracy: 0.7782
Epoch 5/10
215/215 [==============================] - 2s 9ms/step - loss: 0.1108 - accuracy: 0.9559 - val_loss: 0.7728 - val_accuracy: 0.7756
Epoch 6/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0880 - accuracy: 0.9650 - val_loss: 0.8448 - val_accuracy: 0.7677
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss: 0.071

In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[2.1672444e-03],
        [6.4135349e-01],
        [9.9998641e-01],
        [3.9899984e-01],
        [1.4892033e-06],
        [9.9997211e-01],
        [9.0488607e-01],
        [9.9999750e-01],
        [9.9999332e-01],
        [2.5730237e-01]], dtype=float32))

In [47]:
#Convert model prediction probabilities to label format so <0.5 is 0(not disaster) and >0.5 is 1(disaster)
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 76.50918635170603,
 'precision': 0.7654719672276178,
 'recall': 0.7650918635170604,
 'f1': 0.7636361523198555}

In [49]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.27, New accuracy: 76.51, Difference: -2.76
Baseline precision: 0.81, New precision: 0.77, Difference: -0.05
Baseline recall: 0.79, New recall: 0.77, Difference: -0.03
Baseline f1: 0.79, New f1: 0.76, Difference: -0.02


### Model 3: GRU
The GRU cell has similar features to an LSTM cell but has less parameters.

The architecture of the GRU-powered model:

* Input (text) -> Tokenize -> Embedding -> Layers -> Output (label probability)


In [50]:
tf.random.set_seed(42)
from tensorflow.keras import layers

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [51]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [52]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                       

In [53]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20240618-014606
Epoch 1/10
215/215 [==============================] - 11s 38ms/step - loss: 0.5299 - accuracy: 0.7275 - val_loss: 0.4497 - val_accuracy: 0.7808
Epoch 2/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3236 - accuracy: 0.8694 - val_loss: 0.4938 - val_accuracy: 0.7848
Epoch 3/10
215/215 [==============================] - 2s 9ms/step - loss: 0.2204 - accuracy: 0.9159 - val_loss: 0.5539 - val_accuracy: 0.7730
Epoch 4/10
215/215 [==============================] - 2s 10ms/step - loss: 0.1559 - accuracy: 0.9453 - val_loss: 0.6311 - val_accuracy: 0.7874
Epoch 5/10
215/215 [==============================] - 4s 17ms/step - loss: 0.1150 - accuracy: 0.9602 - val_loss: 0.6567 - val_accuracy: 0.7703
Epoch 6/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0862 - accuracy: 0.9695 - val_loss: 0.8285 - val_accuracy: 0.7664
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss: 0.062

In [54]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


((762, 1),
 array([[1.8804544e-03],
        [6.7034674e-01],
        [9.9991345e-01],
        [4.4705261e-02],
        [9.9363991e-05],
        [9.9965274e-01],
        [9.2184329e-01],
        [9.9990880e-01],
        [9.9991238e-01],
        [1.9722911e-02]], dtype=float32))

In [55]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [56]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.11548556430446,
 'precision': 0.7645775884027154,
 'recall': 0.7611548556430446,
 'f1': 0.7579576679636106}

In [57]:
# Compare to baseline
compare_baseline_to_new_results(baseline_results, model_3_results)

Baseline accuracy: 79.27, New accuracy: 76.12, Difference: -3.15
Baseline precision: 0.81, New precision: 0.76, Difference: -0.05
Baseline recall: 0.79, New recall: 0.76, Difference: -0.03
Baseline f1: 0.79, New f1: 0.76, Difference: -0.03


### Model 4: Bidirectional RNN
A standard RNN will process a sequence from left to right, where as a bidirectional RNN will process the sequence from left to right and then again from right to left.

In practice, many sequence models often see and improvement in performance when using bidirectional RNN's.However, this improvement in performance often comes at the cost of longer training times and increased model parameters (since the model goes left to right and right to left, the number of trainable parameters doubles).

In [58]:
tf.random.set_seed(42)
from tensorflow.keras import layers

model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
x = layers.Bidirectional(layers.GRU(64))(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [59]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [60]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 15, 128)           98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                             

In [61]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20240618-014651
Epoch 1/10
215/215 [==============================] - 17s 46ms/step - loss: 0.5064 - accuracy: 0.7494 - val_loss: 0.4538 - val_accuracy: 0.7822
Epoch 2/10
215/215 [==============================] - 3s 14ms/step - loss: 0.3121 - accuracy: 0.8724 - val_loss: 0.5149 - val_accuracy: 0.7730
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.2085 - accuracy: 0.9187 - val_loss: 0.5533 - val_accuracy: 0.7638
Epoch 4/10
215/215 [==============================] - 3s 16ms/step - loss: 0.1417 - accuracy: 0.9534 - val_loss: 0.6776 - val_accuracy: 0.7612
Epoch 5/10
215/215 [==============================] - 3s 14ms/step - loss: 0.1042 - accuracy: 0.9651 - val_loss: 0.7002 - val_accuracy: 0.7507
Epoch 6/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0776 - accuracy: 0.9721 - val_loss: 0.8070 - val_accuracy: 0.7546
Epoch 7/10
215/215 [==============================] - 3s 12ms/s

In [62]:
## Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[8.0461177e-05],
       [6.9858932e-01],
       [9.9998462e-01],
       [3.3768788e-01],
       [2.0205163e-05],
       [9.9996829e-01],
       [9.9862397e-01],
       [9.9999368e-01],
       [9.9999416e-01],
       [1.5329989e-03]], dtype=float32)

In [63]:
#Convert prediction probabilities to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [64]:
#Model results
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 74.93438320209974,
 'precision': 0.7509852568526685,
 'recall': 0.7493438320209974,
 'f1': 0.7467057648828105}

In [65]:
#Compare to baseline
compare_baseline_to_new_results(baseline_results, model_4_results)

Baseline accuracy: 79.27, New accuracy: 74.93, Difference: -4.33
Baseline precision: 0.81, New precision: 0.75, Difference: -0.06
Baseline recall: 0.79, New recall: 0.75, Difference: -0.04
Baseline f1: 0.79, New f1: 0.75, Difference: -0.04


### Model 5 : Conv1D
Convolutional Neural Network for Text
Have use Conv2D for images, but here text which is 1-Dimensional. Hence will use Conv1D.

A typical CNN architecture for sequences will look:

* Inputs (text) -> Tokenization -> Embedding -> Layers -> Outputs (class probabilities)

The difference again is in the layers component.
* Instead of using an LSTM or GRU cell, we're going to use a `tensorflow.keras.layers.Conv1D()` layer followed by a `tensorflow.keras.layers.GlobablMaxPool1D()` layer.

1-dimensional convolving filters are used as ngram detectors, each filter specializing in a closely-related family of ngrams (an ngram is a collection of n-words, for example, an ngram of 5 might result in "hello, my name is Daniel").

Max-pooling over time extracts the relevant ngrams for making a decision.
The rest of the network classifies the text based on this information.

In [66]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sentence into embedding

conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, #looks at 5 words at a time
                        strides = 1, #default
                        padding = 'valid', #drops unecessary ones in valid, keeps it same in 'same'
                        activation="relu") # convolve over target sequence 5 words at a time

conv_1d_output = conv_1d(embedding_test) # pass embedding through 1D convolutional layer

max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get the most important features

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [67]:
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.02552378, -0.00678012, -0.02590024, ...,  0.00720131,
          -0.03262606, -0.01544248],
         [ 0.01960944,  0.03454766,  0.02106217, ..., -0.01844776,
           0.06158143, -0.00050976],
         [ 0.02232702,  0.00178185,  0.02598337, ..., -0.02704928,
           0.01672785, -0.00937902],
         ...,
         [-0.06192297, -0.02888566,  0.0019866 , ..., -0.00899673,
           0.06835619,  0.0472857 ],
         [-0.06192297, -0.02888566,  0.0019866 , ..., -0.00899673,
           0.06835619,  0.0472857 ],
         [-0.06192297, -0.02888566,  0.0019866 , ..., -0.00899673,
           0.06835619,  0.0472857 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[0.09122714, 0.09767839, 0.07524534, 0.04074005, 0.04733073,
          0.        , 0.        , 0.        , 0.00801942, 0.        ,
          0.05979424, 0.        , 0.05109425, 0.        , 0.        ,
          0.00307656, 0.

In [68]:
tf.random.set_seed(42)

from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_5')

inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# summary of 1D convolution model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 32)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 1)              

In [69]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240618-014737
Epoch 1/10
215/215 [==============================] - 9s 36ms/step - loss: 0.5624 - accuracy: 0.7214 - val_loss: 0.4720 - val_accuracy: 0.7848
Epoch 2/10
215/215 [==============================] - 2s 9ms/step - loss: 0.3413 - accuracy: 0.8606 - val_loss: 0.4666 - val_accuracy: 0.7927
Epoch 3/10
215/215 [==============================] - 2s 8ms/step - loss: 0.2149 - accuracy: 0.9199 - val_loss: 0.5300 - val_accuracy: 0.7756
Epoch 4/10
215/215 [==============================] - 1s 7ms/step - loss: 0.1389 - accuracy: 0.9552 - val_loss: 0.6154 - val_accuracy: 0.7795
Epoch 5/10
215/215 [==============================] - 2s 8ms/step - loss: 0.0982 - accuracy: 0.9676 - val_loss: 0.6594 - val_accuracy: 0.7848
Epoch 6/10
215/215 [==============================] - 1s 6ms/step - loss: 0.0829 - accuracy: 0.9723 - val_loss: 0.6928 - val_accuracy: 0.7664
Epoch 7/10
215/215 [==============================] - 2s 11ms/step - loss: 0.067

In [70]:
# Make predictions with model_5
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.37651283],
       [0.99530953],
       [0.99991   ],
       [0.03616539],
       [0.00235876],
       [0.9873197 ],
       [0.7973819 ],
       [0.99967825],
       [0.9999106 ],
       [0.12848453]], dtype=float32)

In [71]:
# Convert model_5 prediction probabilities to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [72]:
# Calculate model_5 evaluation metrics
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 77.95275590551181,
 'precision': 0.7874927923925475,
 'recall': 0.7795275590551181,
 'f1': 0.775282378883342}

In [73]:
# Compare model_5 results to baseline
compare_baseline_to_new_results(baseline_results, model_5_results)

Baseline accuracy: 79.27, New accuracy: 77.95, Difference: -1.31
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.78, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.01


### Model 6: Use Tensorflow Hub Pretrained Sentence Encoder
Multiple pre-trianed models in Tensorfow HUb to choose from, gonna take `Universal Sentence Encoder(USE)`.

The main difference between the embedding layer we create and the Universal Sentence Encoder is that rather than create a word-level embedding, the Universal Sentence Encoder, creates a whole sentence-level embedding.

Our embedding layer also outputs an a 128 dimensional vector for each word, where as, the Universal Sentence Encoder outputs a 512 dimensional vector for each sentence.

In [80]:
import tensorflow_hub as hub
sample_sentence = "There's a flood in my street!"

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2")

embed_samples = embed([sample_sentence,
                       "When you can the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])


tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [81]:
embed_samples[0].shape

TensorShape([512])

In [82]:
sentence_encoder_layer= hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2",
                                       input_shape = [], # can take any input size
                                       dtype = tf.string,
                                       trainable = False,
                                       name = 'USE')

In [108]:
#Create the model
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')],
                              name='model_6_USE')

#Compile the model
model_6.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_14 (Dense)            (None, 128)               65664     
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 256863617 (979.86 MB)
Trainable params: 65793 (257.00 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


The number of paramters in the USE layer, these are the pretrained weights its learned on various text sources (Wikipedia, web news, web question-answer forums, etc, see the Universal Sentence Encoder paper for more).

The trainable parameters are only in our output layers, in other words, we're keeping the USE weights frozen and using it as a feature-extractor. We could fine-tune these weights by setting trainable=True when creating the hub.KerasLayer instance.

In [109]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20240618-023236
Epoch 1/10
215/215 [==============================] - 6s 14ms/step - loss: 0.4883 - accuracy: 0.7900 - val_loss: 0.4437 - val_accuracy: 0.7992
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.4092 - accuracy: 0.8161 - val_loss: 0.4313 - val_accuracy: 0.8150
Epoch 3/10
215/215 [==============================] - 3s 15ms/step - loss: 0.3933 - accuracy: 0.8266 - val_loss: 0.4277 - val_accuracy: 0.8150
Epoch 4/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3835 - accuracy: 0.8311 - val_loss: 0.4234 - val_accuracy: 0.8189
Epoch 5/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3737 - accuracy: 0.8386 - val_loss: 0.4260 - val_accuracy: 0.8189
Epoch 6/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3629 - accuracy: 0.8410 - val_loss: 0.4201 - val_accuracy: 0.8241
Epoch 7/10
215/215 [==============================] - 3s 1

In [110]:
# Make predictions with model_6
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 9ms/step


array([[0.09694009],
       [0.7272093 ],
       [0.9985996 ],
       [0.20085473],
       [0.59304655],
       [0.82520926],
       [0.9884404 ],
       [0.9959053 ],
       [0.96716475],
       [0.06351198]], dtype=float32)

In [111]:
# Convert model_6 prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [112]:
# Calculate model_6 evaluation metrics
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 82.80839895013123,
 'precision': 0.833035534864596,
 'recall': 0.8280839895013123,
 'f1': 0.8260626202685188}

In [113]:
# Compare model_6 results to baseline
compare_baseline_to_new_results(baseline_results, model_6_results)

Baseline accuracy: 79.27, New accuracy: 82.81, Difference: 3.54
Baseline precision: 0.81, New precision: 0.83, Difference: 0.02
Baseline recall: 0.79, New recall: 0.83, Difference: 0.04
Baseline f1: 0.79, New f1: 0.83, Difference: 0.04


### Model 7: Testing how it works on 10% of the data

In [114]:
train_sentences_90_percent, train_sentences_10_percent, train_labels_90_percent, train_labels_10_percent = train_test_split(np.array(train_sentences),
                                                                                                                            train_labels,
                                                                                                                            test_size=0.1,
                                                                                                                            random_state=42)

In [115]:
# Check length of 10 percent datasets
print(f"Total training examples: {len(train_sentences)}")
print(f"Length of 10% training examples: {len(train_sentences_10_percent)}")

Total training examples: 6851
Length of 10% training examples: 686


In [116]:
# Clone model_6 but reset weights
model_7 = tf.keras.models.clone_model(model_6)

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# summary
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_14 (Dense)            (None, 128)               65664     
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 256863617 (979.86 MB)
Trainable params: 65793 (257.00 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [117]:
# Fit the model to 10% of the training data
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=10,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "10_percent_data")])

Saving TensorBoard log files to: model_logs/10_percent_data/20240618-024156
Epoch 1/10
22/22 [==============================] - 7s 47ms/step - loss: 0.6636 - accuracy: 0.6895 - val_loss: 0.6340 - val_accuracy: 0.7205
Epoch 2/10
22/22 [==============================] - 1s 26ms/step - loss: 0.5627 - accuracy: 0.8149 - val_loss: 0.5584 - val_accuracy: 0.7546
Epoch 3/10
22/22 [==============================] - 0s 22ms/step - loss: 0.4692 - accuracy: 0.8324 - val_loss: 0.5051 - val_accuracy: 0.7769
Epoch 4/10
22/22 [==============================] - 1s 26ms/step - loss: 0.4065 - accuracy: 0.8440 - val_loss: 0.4858 - val_accuracy: 0.7756
Epoch 5/10
22/22 [==============================] - 1s 25ms/step - loss: 0.3686 - accuracy: 0.8571 - val_loss: 0.4781 - val_accuracy: 0.7822
Epoch 6/10
22/22 [==============================] - 1s 32ms/step - loss: 0.3392 - accuracy: 0.8659 - val_loss: 0.4741 - val_accuracy: 0.7808
Epoch 7/10
22/22 [==============================] - 1s 41ms/step - loss: 0.315

In [118]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.16552185],
       [0.9575136 ],
       [0.98501366],
       [0.16155416],
       [0.40309745],
       [0.9265652 ],
       [0.94795173],
       [0.9125678 ],
       [0.9177186 ],
       [0.04888037]], dtype=float32)

In [119]:
# Convert prediction probabilities to labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [120]:
# Calculate model results
model_7_results = calculate_results(val_labels, model_7_preds)
model_7_results

{'accuracy': 78.74015748031496,
 'precision': 0.7900609457201325,
 'recall': 0.7874015748031497,
 'f1': 0.7852275674790494}

In [121]:
# Compare to baseline
compare_baseline_to_new_results(baseline_results, model_7_results)

Baseline accuracy: 79.27, New accuracy: 78.74, Difference: -0.52
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.79, Difference: -0.00
